# Google Calendar Events Downloader

* Download all of your google calendar events history and export into csv. 
* For simple data analysis and visualization, see: [google_calendar_data_analysis](https://github.com/markwk/qs_ledger/tree/master/google_calendar/google_calendar_data_analysis.ipynb)

----

## Google Calendar API Docs

* https://developers.google.com/calendar/v3/reference/  
* https://developers.google.com/resources/api-libraries/documentation/calendar/v3/python/latest/

------

# Initial Install and Setup

Follow Steps here: https://developers.google.com/calendar/quickstart/python

1. Click Button "ENABLE THE GOOGLE CALENDAR API"
2. Select + Create a new project.
3. Download the configuration file.
4. Move the downloaded file to your working directory and ensure it is named credentials.json.
5. In terminal or command line, navigate to your project's working directory
6. Run `python quickstart.py`

Afterwards:

> The sample will attempt to open a new window or tab in your default browser. If this fails, copy the URL from the console and manually open it in your browser.

> If you are not already logged into your Google account, you will be prompted to log in. If you are logged into multiple Google accounts, you will be asked to select one account to use for the authorization.

> Click the Accept button.
> The sample will proceed automatically, and you may close the window/tab.

-----

## Libraries and Dependencies

In [1]:
from __future__ import print_function
import datetime
from googleapiclient.discovery import build
from httplib2 import Http
from oauth2client import file, client, tools

import pandas as pd

----

## Authorization Check

In [2]:
# If modifying these scopes, delete the file token.json.
SCOPES = 'https://www.googleapis.com/auth/calendar.readonly'

In [3]:
def authorize():
    store = file.Storage('token.json')
    creds = store.get()
    if not creds or creds.invalid:
        flow = client.flow_from_clientsecrets('credentials.json', SCOPES)
        creds = tools.run_flow(flow, store)
    global service
    service = build('calendar', 'v3', http=creds.authorize(Http()))

In [4]:
authorize()

----

## Quick Start Test in Jupyter

In [5]:
def test():
    """Shows basic usage of the Google Calendar API.
    Prints the start and name of the next 10 events on the user's calendar.
    """

    authorize()
    
    # Call the Calendar API
    now = datetime.datetime.utcnow().isoformat() + 'Z' # 'Z' indicates UTC time
    print('Getting the upcoming 10 events')
    events_result = service.events().list(calendarId='primary', timeMin=now,
                                        maxResults=10, singleEvents=True,
                                        orderBy='startTime').execute()
    events = events_result.get('items', [])

    if not events:
        print('No upcoming events found.')
    for event in events:
        start = event['start'].get('dateTime', event['start'].get('date'))
        print(start, event['summary'])

In [6]:
test()

Getting the upcoming 10 events
2018-12-29T09:30:00+07:00 Writing
2018-12-29T09:30:00+07:00 Free Reading 
2018-12-29T11:00:00+07:00 Meditate
2018-12-29T14:45:00+07:00 Design Hour
2018-12-29T16:45:00+07:00 Int3c Business Check-in & Biz Dev
2018-12-29T20:15:00+07:00 Date with Data HOUR
2018-12-30T10:00:00+07:00 Weekly Review
2018-12-30T11:30:00+07:00 Read: Tickler File
2018-12-31T08:30:00+07:00 Writing
2019-01-01T11:30:00+07:00 Monthly Review


----

## Get List User Calendars

In [7]:
calendars_list = service.calendarList().list().execute()
calendars_dict = calendars_list.get('items', [])

if not calendars_dict:
    print('No calendars found.')

print("Printing Calendar Names...")

calendar_ids = []

for calendar in calendars_dict:
    print(calendar['summary'])
    # create a list of calendar ids
    calendar_ids.append(calendar['id'])

Printing Calendar Names...
Toggl Time
Articles from Pocket
Tech / Startup Meeetings and Events
Avalability
Fitness Calendar
Todoist
webcal://www.trainingpeaks.com/ical/F2VHISIR72BH4.ics
Personal Calendar
Int3c Web Dev Appointments
Moves Export
Habits
https://app.velohero.com/ical/workouts?private_key=wKBDat9VNOGQlI6CBcyNXtxvMC5q9W1ph5YUMzKG
Family Birthdays
Todoist
SportTracks
Theme Day
Panda Half
Contacts
Holidays in China
Week Numbers


In [8]:
print("You have " + str(len(calendar_ids)) + " calendars in this account's Google Calendar")

You have 20 calendars in this account's Google Calendar


In [9]:
# dataframing calendars data
calendars = pd.DataFrame.from_dict(calendars_dict)

In [10]:
calendars.to_csv('data/calendars_list.csv')

----

## Get Past Events from Calendar

In [11]:
def get_calendar_events(calendar_id):
    beginning_time = "2004-01-01T00:00:00.356167Z"
    now = datetime.datetime.utcnow().isoformat() + 'Z' # 'Z' indicates UTC time

    page_count = 0
    events_list = []

    print('Getting all of the past events')

    events = service.events()
    request = events.list(
        #calendarId = 'primary',
        calendarId = calendar_id,
        timeMin=beginning_time,
        timeMax=now,
        maxResults=2500,
        singleEvents=True,
        # showDeleted=True,
        orderBy='updated')

    while request is not None:
        
      page_count = page_count + 1
      print("Getting " + str(page_count) + " page of events...")
      eventsResult = request.execute()

      # Do something with the activities
      print(len(eventsResult.get('items', [])))
      events_list = events_list + list(eventsResult.get('items', []))

      request = events.list_next(request, eventsResult)
    
    return events_list

In [13]:
# generate csv of all calendars in google calendar
for i in calendar_ids:
    print("Printing events for calendar: " + str(i))
    events_list = get_calendar_events(i)
    event_df = pd.DataFrame(events_list)
    filename = str(i) + ".csv"
    filepath = 'data/' + filename
    print("Creating CSV at " + filepath)
    event_df.to_csv(filepath, index=False)

Printing events for calendar: 9u1e7p4lcnj7mlmfgps9adjj6s@group.calendar.google.com
Getting all of the past events
Getting 1 page of events...
1284
Creating CSV at data/9u1e7p4lcnj7mlmfgps9adjj6s@group.calendar.google.com.csv
Printing events for calendar: 4n6mispi9os3t9kd1ec1ednp50@group.calendar.google.com
Getting all of the past events
Getting 1 page of events...
1552
Creating CSV at data/4n6mispi9os3t9kd1ec1ednp50@group.calendar.google.com.csv
Printing events for calendar: int3c.com_svrm8irmhghrrbvamgit3q5qf4@group.calendar.google.com
Getting all of the past events
Getting 1 page of events...
84
Creating CSV at data/int3c.com_svrm8irmhghrrbvamgit3q5qf4@group.calendar.google.com.csv
Printing events for calendar: n213rt5unu15rdmfibv81sf8ik@group.calendar.google.com
Getting all of the past events
Getting 1 page of events...
30
Creating CSV at data/n213rt5unu15rdmfibv81sf8ik@group.calendar.google.com.csv
Printing events for calendar: clckbnn2qm0uiu67b18enbcr40@group.calendar.google.com
G